In [71]:
import pandas as pd
import numpy as np
from readability import Readability
from tqdm import tqdm
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 108 ms


In [2]:
patent_descriptions_df = pd.read_csv("data/patent_descriptions.csv", index_col=0)

time: 21.1 s


In [38]:
decisions_df = pd.read_csv("data/ptab2_decisions.csv")

/Users/JasonKatz/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,9,11,12,13,14,15,16,17,19,20,21,22,23,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


time: 1.88 s


In [39]:
proceedings_df = pd.read_csv("data/ptab2_proceedings.csv")

time: 261 ms


In [40]:
ignore_categories = ['Administrative Remand', 'Affirmed', 'Affirmed and Remanded', 'Affirmed with New Ground of Rejection   ', 
                'Affirmed with new ground of rejection   ', 'Affirmed-in-Part', 'Affirmed-in-Part and Remanded', 
                'Affirmed-in-part and remanded with new ground of rejection   ', 'Affirmed-in-part with new ground of rejection  ', 
                'Inter partes reexam affirmed', 'Inter partes reexam affirmed-in-part', 'Inter partes reexam new ground of rejection', 
                'Inter partes reexam rehearing decision is a new decision', 'Inter partes reexam remand ', 'Inter partes reexam reversed', 
                'Panel Remand', 'Reexam affirmed', 'Reexam affirmed-in-part', 'Reexam affirmed-in-part with new ground of rejection  ', 
                'Reexam rehearing decision final and appealable', 'Reexam rehearing decision is a new decision', 'Reexam reversed', 
                'Request for Adverse Judgment after Institution', 'Request for Adverse Judgment before Institution', 'Reversed', 
                'Reversed and Remanded', 'Reversed with New Ground of Rejection  ', 'Reversed with new ground of rejection  ', 'Vacated', 
                'Vacated and Remanded', 'Vacated with new ground of rejection   ', 'Vacated-in-Part', 'Termination']

good_claim_categories = ['Denied (Patent Owner)', 'Final Decision', 'Final decision', 'Granted', 'Granted (Patent Owner)', 'Granted (Petitioner)', 
                         'Granted-In-Part', 'Granted-in-Part (Patent Owner)', 'Granted-in-Part (Petitioner)', 'Settlement', 'Settlement after Institution', 
                         'Dismissed after Institution']

bad_claim_categories = ['Decision on Petition - Denied', 'Denied (Petitioner)', 'Dismissed', 'Dismissed before Institution', 
                        'Institution Denied', 'Settlement before Institution']

time: 6.94 ms


In [41]:
decisions_df = decisions_df[~decisions_df['subdecision_type_category'].isin(ignore_categories)]

decisions_df['good_claim'] = decisions_df['subdecision_type_category'].isin(good_claim_categories).astype(int)

time: 205 ms


In [43]:
proceedings_df['accorded_filing_date'] = pd.to_datetime(proceedings_df['accorded_filing_date'], errors='coerce')
proceedings_df['respondent_grant_date'] = pd.to_datetime(proceedings_df['respondent_grant_date'], errors='coerce')
proceedings_df = proceedings_df[proceedings_df['accorded_filing_date'].between('1800-01-01', pd.datetime.today())]
proceedings_df = proceedings_df[proceedings_df['respondent_grant_date'].between('1800-01-01', pd.datetime.today())]
proceedings_df['days_between_grant_and_filing'] = (proceedings_df['accorded_filing_date'] - proceedings_df['respondent_grant_date']).dt.days.astype('int16')

time: 183 ms


In [62]:
training_data = pd.DataFrame()

time: 130 ms


In [63]:
training_data['proceeding_number'] = decisions_df['proceeding_number']
training_data['patent_id'] = decisions_df['respondent_patent_number']
training_data['good_claim'] = decisions_df['good_claim']

time: 11.7 s


In [65]:
training_data = training_data.merge(patent_descriptions_df, on="patent_id", how="left")

time: 3.72 s


In [66]:
training_data = training_data.merge(proceedings_df[['proceeding_number', 'days_between_grant_and_filing']], on="proceeding_number", how="left")

time: 109 ms


In [60]:
# training_data['detailed_description'] = training_data['detailed_description'].fillna("")

time: 15.8 ms


In [67]:
training_data['description_word_count'] = training_data['detailed_description'].str.split().str.len()

time: 34.5 s


In [69]:
training_data['description_word_count'] = training_data['description_word_count'].fillna(training_data['description_word_count'].median())

time: 298 ms


In [ ]:
fig_counts = []
fk_scores = []
for description in tqdm(training_data['detailed_description']):
    try:
        description = description.lower()
        r = Readability(description.replace('aed-512', ''))
        fk_score = r.flesch_kincaid().score
    except:
        fk_scores.append(np.nan)
        fig_counts.append(np.nan)
        continue
    fk_scores.append(fk_score)
    fig_counts.append(description.count("fig.") + description.count("figs."))
training_data['fig_counts'] = fig_counts
training_data['fk_score'] = fk_scores

  8%|▊         | 427/5613 [02:10<27:03,  3.19it/s]

In [ ]:
training_data['fk_score'] = training_data['fk_score'].fillna(training_data['fk_score'].median())
training_data['fig_counts'] = training_data['fig_counts'].fillna(training_data['fig_counts'].median())

In [ ]:
training_data.to_csv("data/training_data.csv")